# Razpoznava točk daljnovodov

## Krmilni parametri

V tem odseku se nahajajo krmilni parametri aplikacije:
* **ASSETS_FOLDER** - v tem direktoriju se nahajajo *.las* datoteke
* **OUTPUT_FOLDER** - v ta direktorij se shranjujejo izhodne *.las* datoteke
* **FILENAME** - ime vhodne datoteke, mora se nahajati v **ASSETS_FOLDER**
* **OUTPUT_FILENAME** - ime izhodne datoteke
* **THR** - maksimalna dovoljena razdalja od premice najboljšega prileganja
* **K** - število ponovitev algoritma

Vsi direktoriji morajo obstajati za pravilno delovanje aplikacije.

In [ ]:
import laspy as las
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skspatial.objects import Line, Point, Points
from skspatial.plotting import plot_3d

ASSETS_FOLDER = 'assets/'
OUTPUT_FOLDER = 'output/'
FILENAME = 'GK_559_146.las'
OUTPUT_FILENAME = 'output'
THR = 10
K = 1000

Funkcija, ki prikaže naključen nabor točk in njihovo najboljše prilegajoče se premico.

In [ ]:
def plot_points(points, line):
  plot_3d(
    points.plotter(c='r', s=10, depthshade=False),
    line.plotter(t_1=-50, t_2=50),
  )

## Filtriranje točk

Iz *.las* datoteke se podatki o *X*, *Y*, *Z* in *classification* združijo v seznam, pridobi se tudi minimalen *Z*, ki označuje višino.

Testiranje je dalo najboljše rezultate, ko se filtrirajo točke, ki so na višini $ minimum * 1.08 $ ali višje. 

Izračuna se naključen *N*, ki je med *0.05%* in *2%*, točke se preuredijo v naključnem vrstnem redu in izbere se prvih *N* točk, zabeleži pa se tudi preostanek.

Na koncu se še izpišejo *N*, dolžina filtriranih točk, dolžina vseh točk in $ n-točk + preostanek + manjše\;od\;minimuma $

In [ ]:
def get_points(file):
  print('GETTING POINTS')
  points = np.vstack((file.X, file.Y, file.Z, file.classification)).transpose()

  minimum = min(file.Z)
  print(minimum, max(file.Z), minimum * 1.08)

  #filtered_points = [point for point in points if point[3] < 2]
  #remaining_points = [point for point in points if point[3] >= 2]
  filtered_points = [point for point in points if point[2] > (minimum * 1.08)]
  remaining_points = [point for point in points if point[2] <= (minimum * 1.08)]

  #remaining_points.extend([point for point in filtered_points if point[2] <= (minimum + 1100)])
  #filtered_points = [point for point in filtered_points if point[2] > (minimum + 1100)]

  N = int(len(filtered_points) * random.uniform(0.0005, 0.02))
  random.shuffle(filtered_points)

  if N < 2:
    N = 2
  
  N_points = filtered_points[0:N]
  other_points = filtered_points[N:]

  print(N, len(filtered_points), len(points), len(N_points) + len(remaining_points) + len(other_points))

  return filtered_points, N_points, other_points, remaining_points

## RANSAC

Funkcija kot parametre sprejme *N_points*, ki je majhen nabor izven *filtered_points*, *other_points* pa je preostanek, zadnji parameter pa je *error*, ki opisuje napako.

Vsaka koordinata v točkah je v tem koraku deljena s 1000, kajti brez tega koraka funkcija *best_fit* vrača slabše rezultate.

Iz *N_points* se izračuna najboljše prilegajoča se premica, potem pa se iterira skozi preostale točke *other_points*, kjer se za vsako točko izračuna razdalja do premice. Če je ta razdalja manjša kot **THR**, potem se ta točka odstrani iz preostalih točk *other_points* in se doda k *N_points*.

Izpiše se razmerje *N_points* in vseh filtriranih točk *filtered_points* in če je to razmerje nad *0.09* oz. je vseh *N_točk* 9% izmed vseh filtriranih točk, potem se izračuna ponovna najboljše prilegajoče se premica in povprečna napaka, če je napaka manjša od trenutne najboljše napake se tudi vrne iz funkcije napaka, *N_points* in preostale točke *new_other_points*.

In [ ]:
def fit(N_points, other_points, filtered_points, error):
  print('FITTING')
  x = list(map(lambda point: point[0] / 1000, N_points))
  y = list(map(lambda point: point[1] / 1000, N_points))
  z = list(map(lambda point: point[2] / 1000, N_points))

  points_array = Points(np.column_stack((x, y, z)))
  line_fit = Line.best_fit(points_array)
  new_other_points = []

  plot_points(points_array, line_fit)

  for point in list(other_points):
    distance = line_fit.distance_point(Point([point[0] / 1000, point[1] / 1000, point[2] / 1000]))

    if(distance < THR):
      N_points.append(point)
    else:
      new_other_points.append(point)

  print(len(N_points) / len(filtered_points))

  if len(N_points) / len(filtered_points) > 0.09:
    print('FOUND FIT')
    x = list(map(lambda point: point[0] / 1000, N_points))
    y = list(map(lambda point: point[1] / 1000, N_points))
    z = list(map(lambda point: point[2] / 1000, N_points))

    points_array = Points(np.column_stack((x, y, z)))
    line_fit = Line.best_fit(points_array)
    err = 0.0

    for point in N_points:
      err += line_fit.distance_point([point[0] / 1000, point[1] / 1000, point[2] / 1000])

    err = err / len(N_points)

    print('ERRORS CALCULATED')
    if(err < error):
      return err, N_points, new_other_points

  return None, None, None

## Glavni del programa

Prebere se *.las* datoteka, nato pa se **K**-krat pridobijo naključne točke in algoritem RANSAC. V primeru, da funkcija **fit** vrne boljše rezultate kot v prejšnih iteracijah, se bo shranla nova datoteka.

In [ ]:
file = las.read(ASSETS_FOLDER + FILENAME)
error = np.iinfo(np.int64).max

for k in range(K):
  print("===================================")
  print(F"K = {k + 1}")
  filtered_points, N_points, other_points, remaining_points = get_points(file)

  new_err, N_points, other_points = fit(N_points, other_points, filtered_points, error)

  if new_err != None and N_points != None and other_points != None:
    error = new_err
    filename = OUTPUT_FOLDER + OUTPUT_FILENAME + " " + str(error) + ".las"

    print(f'SAVING TO FILE: {filename}')

    all_points = list(map(lambda point: [point[0], point[1], point[2], 16], N_points)) + other_points + remaining_points

    output_file = las.LasData(file.header)
    output_file.points = file.points.copy()
    output_file.X = list(map(lambda point: point[0], all_points))
    output_file.Y = list(map(lambda point: point[1], all_points))
    output_file.Z = list(map(lambda point: point[2], all_points))
    output_file.classification = list(map(lambda point: point[3], all_points))

    output_file.write(filename)

## Rezultati

Na spodnji sliki je po približno 70 iteracijah prikazan zaznan daljnovod.

![Zaznani daljnovodi](docs/output.JPG)

Na spodnji sliki je prikazana celotna *.las* datoteka.

![Celotna slika](docs/entire.JPG)

Na spodnji sliki pa je še prikazan odsek in dokaz klasifikacije.

![Označeno](docs/entire-marked.JPG)